In [6]:
import torch
import numpy as np
from stable_baselines3 import PPO


import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_move_sector  import HumanMoveSectorAction
import common.mlflow_sb3_helper as mlf

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': {
        'activation': 'ReLU',
        'pi': [256, 256],
        'qf': [256, 256],
        'vf': [256, 256],
    },
    'training': {
        'iteration_count': 1,
        'episode_count': 39000,
        'policy': 'MlpPolicy',
        'learning_rate': 0.001,
        'buffer_size': 1500000,
        'learning_starts': 100,
        'batch_size': 256,
        'tau': 0.001,
        'gamma': 0.95,
        'verbose': 0,
        'device': device,
    },
    'options':{
        'finish_dist':100,
        'start_dist':110,
        'delta_phi':0
    },
    'validation': {
        'validate_agent_every_n_eps': 10000,
        'log_interval': 10
    },
    'evaluation': {
        'episode_count': 1
    }
}


option_var ={
    #'PPO 100_0':{'finish_dist':100, 'start_dist':110, 'delta_phi':0 },
    #'PPO 80_0':{'finish_dist':80, 'start_dist':110, 'delta_phi':0 },
    #'PPO 60_0':{'finish_dist':60, 'start_dist':110, 'delta_phi':0 },
    #'PPO 40_0':{'finish_dist':40, 'start_dist':110, 'delta_phi':0 },
    'PPO 20_0':{'finish_dist':20, 'start_dist':110, 'delta_phi':0 },
    'PPO 10_0':{'finish_dist':10, 'start_dist':110, 'delta_phi':0 },
    #'PPO 10_10':{'finish_dist':10, 'start_dist':110, 'delta_phi':10 },
    #'PPO 10_20':{'finish_dist':10, 'start_dist':110, 'delta_phi':20 },
    #'PPO 10_30':{'finish_dist':10, 'start_dist':110, 'delta_phi':30 },
    #'PPO 10_40':{'finish_dist':10, 'start_dist':110, 'delta_phi':40 },
}

option_var2 ={
    'PPO 100_0':{'finish_dist':100, 'start_dist':110, 'delta_phi':0 },
    'PPO 80_0':{'finish_dist':80, 'start_dist':110, 'delta_phi':0 },
    'PPO 80_20':{'finish_dist':80, 'start_dist':110, 'delta_phi':20 },
    'PPO 80_45':{'finish_dist':80, 'start_dist':110, 'delta_phi':45 },
    'PPO 80_90':{'finish_dist':80, 'start_dist':110, 'delta_phi':90 },
    'PPO 80_135':{'finish_dist':80, 'start_dist':110, 'delta_phi':135 },
    'PPO 80_180':{'finish_dist':80, 'start_dist':110, 'delta_phi':180 },
    'PPO 60_180':{'finish_dist':60, 'start_dist':110, 'delta_phi':180 },
    'PPO 40_180':{'finish_dist':40, 'start_dist':110, 'delta_phi':180 },
    'PPO 20_180':{'finish_dist':20, 'start_dist':110, 'delta_phi':180 },
    'PPO 10_180':{'finish_dist':10, 'start_dist':110, 'delta_phi':180 },
}

In [8]:
options=exp_params['options']
#options=None

env = HumanMoveSectorAction(continuous=True, target_point_rand=False, object_ignore=True, options=options)
env_render = HumanMoveSectorAction(continuous=True, target_point_rand=False, object_ignore=True, render_mode='rgb_array', options=options)


In [9]:

mlflow_server = mlf.MLflowServerHelper("http://192.168.0.206:2670", False)
mlflow_path = {}

In [ ]:
#!!!!! Новый эксперимент
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = mlflow_server.new_experiment(exp_name)


In [ ]:
#!!!!! Дорасчет старого эксперимента
experiment_id = 298
exp_name = mlflow_server.get_experiment(experiment_id)

print(exp_name)

In [ ]:
exp_params['exp_id'] = experiment_id

b_first = True
model = None

for name, option in option_var2.items():


    print(name)

    exp_params['env_name'] = env.name()
    exp_params['algorithm_name'] = name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))
    exp_params['options'] = option


    env.set_options(option)
    env_render.set_options(option)

    if b_first == True:
        b_first = False
        model = PPO(  exp_params['training']['policy'], 
                    env,
                    #policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    batch_size=exp_params['training']['batch_size'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    device=device,
                    )
    else:
        print(exp_name)
        model = PPO.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = mlflow_server.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])

    mlflow_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'




In [ ]:
mlflow_path

In [11]:
mlflow_path = {
 'DQN': 'mlflow-artifacts:/216/684a0b5a74e7466498e7f42b7b84b0c6/artifacts/env_HumanMoveSimple_exp_2009_114432/sb3/model.zip'
 }

In [ ]:
experiment_id = 216
load_path = os.path.join(str(experiment_id))
if  os.path.isdir(load_path) == False:
    os.mkdir(load_path)
load_path

In [ ]:
loadel_models = {}

for model_name, path in mlflow_path.items():
    loadel_models[model_name] =  f'{load_path}/{model_name}/'
    mlflow_server.load_artifact(path,loadel_models[model_name])

In [13]:
exp_params['training']['episode_count'] = 99000

exp_params['options']['finish_dist'] = 10
exp_params['options']['start_dist'] = 110
exp_params['options']['delta_phi'] = 0
options=exp_params['options']

env = HumanMoveSimpleAction(target_point_rand=False,options=options)
env_disc = HumanMoveSimpleAction(continuous=False, target_point_rand=False,options=options)

env_render = HumanMoveSimpleAction(target_point_rand=False, render_mode='rgb_array',options=options)
env_disc_render = HumanMoveSimpleAction(continuous=False, target_point_rand=False, render_mode='rgb_array',options=options)

In [ ]:
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = mlflow_server.new_experiment(exp_name)
exp_params['exp_id'] = experiment_id
print(exp_name)

In [ ]:
#!!! Дорасчет в СТАРОМ эксперименте
exp_name = mlflow_server.get_experiment(experiment_id)
print(exp_name)
exp_params['exp_id'] = experiment_id
exp_params['env_name'] = env.name()

In [15]:

models_exclude = {'PPO':True, 'DQN': False, 'DDPG': True, 'SAC': True, 'TD3': True}
#models_exclude = None

In [ ]:
run_post='_10'
exp_params['training']['DQN'] = eps_var['DQN3']

for name, model_path in loadel_models.items():

    if models_exclude != None and models_exclude[name] == True:
        continue
    print(name)

    exp_params['algorithm_name'] = name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    set_env = env_disc if name == 'DQN' or name == 'PPO' else env
    model = models[name].load(model_path + 'model.zip', env=set_env, device=device)
    
    #model.seed(seed=exp_params['seed'])

    if name == 'DQN':
        model.exploration_fraction=exp_params['training']['DQN']['fraction_eps']
        model.exploration_initial_eps= exp_params['training']['DQN']['start_eps']
        model.exploration_final_eps= exp_params['training']['DQN']['final_eps']
        model._setup_model()

    art_loc, exp_name, run_id = mlflow_server.learn_and_fix(
                                model=model, 
                                env= env_disc_render if name == 'DQN' or name == 'PPO' else env_render,
                                run_name=name + run_post,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])

In [ ]:
shutil.rmtree(os.path.join(load_path))